<a href="https://colab.research.google.com/github/bhaveshsingh0206/Contextual-Embedding-based-Stock-Price-Prediction-using-Bidirectional-LSTM/blob/main/Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install gensim
!pip install sns
!pip install torchvision 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import requests


In [ ]:
text=[]
sentiment=[]
with open('/content/drive/MyDrive/NLP Dataset/FinancialPhraseBank-v1.0 2/Sentences_50Agree.txt', encoding = "ISO-8859-1") as f:
  lines = f.read().splitlines()
  for line in lines:
    line=line.rstrip().split('@')
    text.append(' '.join(line[:-1]))
    sentiment.append(line[-1])

In [ ]:
data = {'Text': text, 'Sentiment': sentiment}  
df = pd.DataFrame(data)  
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.head()

In [ ]:
df['Sentiment'].unique()

In [ ]:
from bs4 import BeautifulSoup
import re

In [ ]:
punctuations = '''!()-[]{};:"\,<>./?@#%^&*_~'''
def cleanText(x):
  x = x.lower().strip()
  soup = BeautifulSoup(x)
  x = soup.get_text()
  x = re.sub(r'https?://\S+', '', x)
  x=re.sub("\s\s+", " ", x.strip())
  no_punct = ""
  for char in x:
    if char not in punctuations:
        no_punct = no_punct + char
    else:
        no_punct += " "
  x = no_punct

  x=re.sub("\s\s+", " ", x.strip())
  if (x==' ' or len(x)==0): 
    return np.nan
  return x

In [ ]:
df['Text']= df['Text'].apply(cleanText)
maxLen = 0
for l in df['Text']:
  maxLen = max(maxLen, len(l.split(' ')))
print(maxLen)

In [ ]:
import torch
import numpy as np
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
labels = {'neutral':0,
          'negative':1,
          'positive':2,
          }

In [ ]:
def change_label(x):
  return labels[x]
df['Sentiment_Label']=df['Sentiment'].apply(change_label)

In [ ]:
# /df.head()

In [ ]:
labels = {'neutral':0,
          'negative':1,
          'positive':2,
          }
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels = [labels[label] for label in df['Sentiment']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                            return_tensors="pt") for text in df['Text']]
    def classes(self):
        return self.labels
    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y

In [ ]:
np.random.seed(112)
df_train, df_val = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.9*len(df))])

print(len(df_train),len(df_val))

In [ ]:
df_train.head()

In [ ]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 3)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        # print(input_id.shape)
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask, return_dict=False)
        dropout_output = self.dropout(pooled_output)
        # print(dropout_output.shape)
        linear_output = self.linear(dropout_output)
        # print(linear_output.shape)
        final_layer = self.relu(linear_output)
        # print(final_layer.shape)
        return final_layer
    
    def get_vector(self, input_id, mask):
      _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask, return_dict=False)
      dropout_output = self.dropout(pooled_output)
      return dropout_output

In [ ]:
# batch, max_seq_len, dimension

In [ ]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                batch_loss = criterion(output, train_label.long())
                # break
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            # break
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  

In [ ]:
EPOCHS = 5
bert_pretrained_model = BertClassifier()                        
LR = 1e-6                                                                                                                       
train(bert_pretrained_model, df_train, df_val, LR, EPOCHS)

In [ ]:
# Save Bert Model
# torch.save(bert_pretrained_model.state_dict(), '/content/drive/MyDrive/NLP Dataset/model/bert_pretrained_model.pt')
bert_pretrained_model = BertClassifier()
bert_pretrained_model.load_state_dict(torch.load('/content/drive/MyDrive/NLP Dataset/model/bert_pretrained_model.pt'))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from torch.optim import Adam
import torch
import random
import torch.nn as nn
import torch.nn.functional as F

class GRUModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(GRUModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.gru = nn.GRU(input_size, hidden_dim, n_layers, batch_first=True, bidirectional=True, dropout=0.2)   
        self.linear = nn.Linear(self.hidden_dim*4 , 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.out = nn.Linear(64, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        h_lstm, _ = self.gru(x, hidden)
        avg_pool = torch.mean(h_lstm, 1)
        max_pool, _ = torch.max(h_lstm, 1)
        conc = torch.cat(( avg_pool, max_pool), 1)
        conc = self.relu(self.linear(conc))
        conc = self.dropout(conc)
        out = self.out(conc)
        return out
    
    def get_vector(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        h_lstm, _ = self.gru(x, hidden)
        avg_pool = torch.mean(h_lstm, 1)
        max_pool, _ = torch.max(h_lstm, 1)
        conc = torch.cat(( avg_pool, max_pool), 1)
        conc = self.linear(conc)
        conc = self.dropout(conc)
        return conc
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers*2, batch_size, self.hidden_dim)
        return hidden.to(device)

In [ ]:
from torch.optim import Adam
import torch
import torch.nn as nn
import torch.nn.functional as F
class LSTMModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True, bidirectional=True, dropout=0.2)   
        self.linear = nn.Linear(self.hidden_dim*4 , 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.out = nn.Linear(64, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)
        hidden, c = self.init_hidden(batch_size)
        h_lstm, _ = self.lstm(x, (hidden, c))
        avg_pool = torch.mean(h_lstm, 1)
        max_pool, _ = torch.max(h_lstm, 1)
        conc = torch.cat(( avg_pool, max_pool), 1)
        conc = self.relu(self.linear(conc))
        conc = self.dropout(conc)
        out = self.out(conc)
        return out
    
    def get_vector(self, x):
        batch_size = x.size(0)
        hidden, c = self.init_hidden(batch_size)
        h_lstm, _ = self.lstm(x, (hidden, c))
        avg_pool = torch.mean(h_lstm, 1)
        max_pool, _ = torch.max(h_lstm, 1)
        conc = torch.cat(( avg_pool, max_pool), 1)
        conc = self.linear(conc)
        conc = self.dropout(conc)
        return conc
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers*2, batch_size, self.hidden_dim)
        c = torch.zeros(self.n_layers*2, batch_size, self.hidden_dim)
        return hidden.to(device), c.to(device)

In [ ]:
import gensim.downloader as api
model = api.load("word2vec-google-news-300")
word2Vec = model

In [ ]:
'Sunday' in word2Vec.wv.vocab

In [ ]:
XSentences = []
import math
for i, sent in enumerate(df['Text'].tolist()):
    sent = sent.split(' ')
    temp = []
    for word in sent:
        if word in word2Vec.wv.vocab:
            temp.append(word2Vec[word])
    if(len(temp)==0):
        temp.append(np.zeros(300))
    XSentences.append(temp)

In [ ]:
print(len(XSentences))

In [ ]:
def createXForRNN(l, XSentences):
    X = np.zeros([len(XSentences), l, 300])
    for i, sent in enumerate(XSentences):
        for j in range(0, l):
            if(j<len(sent)):
                word = sent[j]
                temp=word
            else:
                temp = np.zeros(300)

            X[i][j] = temp
    return X
X_rnn = createXForRNN(maxLen, XSentences)
print(X_rnn.shape)

In [ ]:
from sklearn.model_selection import train_test_split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 2
def createDataLoader(X_train, Y_train, X_test, Y_test):
  X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
  Y_train = torch.tensor(Y_train,  dtype=torch.long).to(device)
  X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
  Y_test = torch.tensor(Y_test,  dtype=torch.long).to(device)
  train = torch.utils.data.TensorDataset(X_train, Y_train)
  test = torch.utils.data.TensorDataset(X_test, Y_test)

  train_loader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test, batch_size=BATCH_SIZE, shuffle=False)
  return train_loader, test_loader

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_rnn, np.array(df['Sentiment_Label'].tolist()), test_size=0.1, random_state=42)
train_loader, test_loader = createDataLoader(X_train, Y_train, X_test, Y_test)
import time
def evaluate(n_epochs, model, optimizer, train_loader, test_loader, criterion):
    def calculateCorrect(t, y):
        sum=0
        for i, v in enumerate(t):
            if(t[i]==y[i]):
                sum+=1
        return sum

    for e in tqdm(range(n_epochs)):
        start_time = time.time()
        model.train()
        avg_loss = 0.  
        for i, (x, y) in enumerate(train_loader):
            y_pred = model(x)
            loss = criterion(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            avg_loss += loss.item() / len(train_loader)
      
        model.eval()        
        avg_val_loss = 0.
        val_preds = np.zeros((len(X_test), 3))
      
        for i, (x_batch, y_batch) in enumerate(test_loader):
            y_pred = model(x_batch).detach()
            avg_val_loss += criterion(y_pred, y_batch).item() / len(test_loader)
            val_preds[i * BATCH_SIZE:(i+1) * BATCH_SIZE] = F.softmax(y_pred).cpu().numpy()
          
        accuracy = calculateCorrect(val_preds.argmax(axis=1), Y_test)/len(Y_test)
        elapsed_time = time.time() - start_time 
        print('Epoch {}/{} \t loss={:.4f} \t test_loss={:.4f}  \t test_acc={:.4f}  \t time={:.2f}s'.format(
                  e + 1, n_epochs, avg_loss, avg_val_loss, accuracy, elapsed_time))

In [ ]:
from torch.optim import Adam
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1 = LSTMModel(300, 3, 20, 2)
optimizer_rnn = Adam(model1.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
model1.to(device)

evaluate(20, model1, optimizer_rnn, train_loader, test_loader)

In [ ]:
# Save LSTM Model
# torch.save(model1.state_dict(), '/content/drive/MyDrive/NLP Dataset/model/lstm.pt')
model1 = LSTMModel(300, 3, 20, 2)
model1.load_state_dict(torch.load('/content/drive/MyDrive/NLP Dataset/model/lstm.pt'))
# model1 = torch.load('/content/drive/MyDrive/NLP Dataset/model/lstm.pt')

In [ ]:
from torch.optim import Adam
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model2 = GRUModel(300, 3, 20, 2)
optimizer2 = Adam(model2.parameters(), lr=0.001)
criterion2 = nn.CrossEntropyLoss()
model2.to(device)

evaluate(20, model2, optimizer2, train_loader, test_loader, criterion2)

In [ ]:
# Save GRU Model
torch.save(model2.state_dict(), '/content/drive/MyDrive/NLP Dataset/model/gru.pt')
# model2 = GRUModel(300, 3, 20, 2)
# model2.load_state_dict(torch.load('/content/drive/MyDrive/NLP Dataset/model/gru.pt'))
# model2 = torch.load('/content/drive/MyDrive/NLP Dataset/model/gru.pt')

Time Series

In [ ]:
import pandas as pd
ts = pd.read_csv('/content/drive/MyDrive/NLP Dataset/choti_bhabhi.csv')

In [ ]:
ts.head()

In [ ]:
ts.isna().sum()

In [ ]:
ts.info()

In [ ]:
print(len(ts['yahoo'][0].split(' ')))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
def createXForTimeSeries(ts, lenX):
  ts['split'] = ts['yahoo'].str.slice(1, -1)
  ts=ts.join(ts['split'].str.split(',', -1, expand=True).astype(float))
  temp = ts[[x for x in range(0, lenX+1)]]
  temp[[x for x in range(0, lenX+1)]] = scaler.fit_transform(temp[[x for x in range(0, lenX+1)]])
  X = temp[[x for x in range(0, lenX)]]
  X = np.array(X)
  X = X.reshape(X.shape[0], X.shape[1], 1)
  Y = ts[[lenX]]
  Y = np.array(Y)
  Y = Y.reshape(Y.shape[0], )
  X_Next = ts[[x for x in range(lenX)]]
  X_Next = np.array(X_Next)
  X_Next = X_Next.reshape(X_Next.shape[0], X_Next.shape[1], 1)
  return ts, X, Y, X_Next

In [ ]:
Y[0]

In [ ]:
import numpy as np
ts, X, Y, X_Next = createXForTimeSeries(ts, 100)
print(X.shape, Y.shape)

In [ ]:
X_Next.shape 

In [ ]:
ts.head()

In [ ]:
ts['Sentiment'] = 'neutral'
ts['Text'] = ts['clean_title']

In [ ]:
ts.head()

In [ ]:
class TimeSeries(nn.Module):
  def __init__(self, input_size, output_size, hidden_dim, n_layers):
    super(TimeSeries, self).__init__()
    self.hidden_dim = hidden_dim
    self.n_layers = n_layers
    self.gru = nn.GRU(input_size, hidden_dim, n_layers, batch_first=True, bidirectional=False, dropout=0.2)   
    self.linear = nn.Linear(self.hidden_dim*self.n_layers , 64)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(0.2)
    self.out1 = nn.Linear(64, 32)
    self.out2 = nn.Linear(32, 1)
  def forward(self, x, news):
    # x-> batch, 100, 1
    # news has to be mapped with hidden state
    # bert mappes it to batch, 768 -> final bert outout
    # num_layers*biredirection, batch, hidden
    news = news.unsqueeze(0)
    x, _ = self.gru(x, news)
    x = x[:, -1, :]
    x = self.relu(self.linear(x))
    x = self.dropout(x)
    x = self.out2(self.relu(self.out1(x)))
    # batch, 1
    return x

In [ ]:
def evaluateTimeSeries(n_epochs, model, optimizer, train_loader, test_loader, criterionT):
    for e in tqdm(range(n_epochs)):
        start_time = time.time()
        model.train()
        avg_loss = 0.  
        for i, (x, news, y, next) in enumerate(train_loader):
            y_pred = model(x, news)
            y=y.unsqueeze(1)
            loss = criterionT(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            avg_loss += loss.item() / len(train_loader)
      
        model.eval()        
        avg_val_loss = 0.
        # val_preds = np.zeros((len(X_test), 1))
      
        for i, (x_batch, news_batch, y_batch, next) in enumerate(test_loader):
            y_pred = model(x_batch, news_batch).detach()
            y_batch=y_batch.unsqueeze(1)
            avg_val_loss += criterionT(y_pred, y_batch).item() / len(test_loader)
            # val_preds[i * BATCH_SIZE:(i+1) * BATCH_SIZE] = F.softmax(y_pred).cpu().numpy()
        elapsed_time = time.time() - start_time 
        print('Epoch {}/{} \t test loss={:.4f} \t time={:.2f}s'.format(
                  e + 1, n_epochs, avg_loss, elapsed_time))

In [ ]:
def createXNews(ts, bertmodel):
  X_news = np.zeros([ts.shape[0], 768])
  count=0
  for news in tqdm(ts['Text'].tolist()):
    data = {'Text': [news], 'Sentiment': ['neutral']}  
    newdf = pd.DataFrame(data)  
    n = Dataset(newdf)
    train_dataloader = torch.utils.data.DataLoader(n, batch_size=1)
    for train_input, _ in train_dataloader:
      mask = train_input['attention_mask'].to(device)
      input_id = train_input['input_ids'].squeeze(1).to(device)
      output = bertmodel.get_vector(input_id, mask)
      X_news[count] = output[0].cpu().detach().numpy()
      count+=1
  return X_news

In [ ]:
XNewsSentences = []
import math
maxSequenceLen = 0
for i, sent in enumerate(ts['Text'].tolist()):
    sent = sent.split(' ')
    maxSequenceLen=max(maxSequenceLen, len(sent))
    temp = []
    for word in sent:
        if word in word2Vec.wv.vocab:
            temp.append(word2Vec[word])
    if(len(temp)==0):
        temp.append(np.zeros(300))
    XNewsSentences.append(temp)
print(maxSequenceLen)
def createXNewsForLSTM(l, XNewsSentences):
    X = np.zeros([len(XNewsSentences), l, 300])
    for i, sent in enumerate(XNewsSentences):
        for j in range(0, l):
            if(j<len(sent)):
                word = sent[j]
                temp=word
            else:
                temp = np.zeros(300)

            X[i][j] = temp
    return X
def createXNewsModelReadyForLSTM(dim, model, maxSequenceLen, XNewsSentences):
  X_News_LSTM = createXNewsForLSTM(maxSequenceLen, XNewsSentences)
  # X_News_LSTM = torch.tensor(X_News_LSTM, dtype=torch.float32).cpu()
  # X_News_LSTM -> total, seq, 300


  X_News = torch.tensor(np.zeros([len(X_News_LSTM), dim]))
  model.eval()
  model.to(device) 
  for i, news in tqdm(enumerate(X_News_LSTM)):
    # news->seq, 300
    news = torch.tensor(news, dtype=torch.float32)
    news = news.unsqueeze(0)
    X_News[i] = model.get_vector(news.to(device))
  return X_News

# print(total, 64)

In [ ]:
y = torch.zeros([100, 100])
# for x in range()

In [ ]:
# T=torch.tensor(X_News_LSTM[0], dtype=torch.float32).unsqueeze(0)

In [ ]:
# def createXNewsForLSTM(ts, model, dim):
#   X_news = np.zeros([ts.shape[0], dim])
#   # count=0
#   for news in tqdm(ts['Text'].tolist()):
#     pass
#   # for news in tqdm(ts['Text'].tolist()):
#   #   data = {'Text': [news], 'Sentiment': ['neutral']}  
#   #   newdf = pd.DataFrame(data)  
#   #   n = Dataset(newdf)
#   #   train_dataloader = torch.utils.data.DataLoader(n, batch_size=1)
#   #   for train_input, _ in train_dataloader:
#   #     mask = train_input['attention_mask'].to(device)
#   #     input_id = train_input['input_ids'].squeeze(1).to(device)
#   #     output = model.get_vector(input_id, mask)
#   #     X_news[count] = output[0].cpu().detach().numpy()
#   #     count+=1
#   return X_news

In [ ]:
def createDataLoaderForNews(X_train, News_train, Y_train, X_test, News_test, Y_test, BATCH_SIZE, X_Next_train, X_Next_test):
  X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
  Y_train = torch.tensor(Y_train,  dtype=torch.float32).to(device)

  News_train = torch.tensor(News_train,  dtype=torch.float32).to(device)
  X_Next_train = torch.tensor(X_Next_train,  dtype=torch.float32).to(device)

  X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
  Y_test = torch.tensor(Y_test,  dtype=torch.float32).to(device)
  News_test = torch.tensor(News_test,  dtype=torch.float32).to(device)
  X_Next_test = torch.tensor(X_Next_test,  dtype=torch.float32).to(device)

  train = torch.utils.data.TensorDataset(X_train, News_train, Y_train, X_Next_train)
  test = torch.utils.data.TensorDataset(X_test, News_test, Y_test, X_Next_test)

  train_loader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test, batch_size=BATCH_SIZE, shuffle=False)
  return train_loader, test_loader

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# X_news = np.zeros([ts.shape[0], 768])
from tqdm import tqdm
# bert_pretrained_model.to(device)
model1.to(device)
X_news = createXNews(ts, bert_pretrained_model)
X_news_LSTM = createXNewsModelReadyForLSTM(64, model1, maxSequenceLen, XNewsSentences)
X_news_GRU = createXNewsModelReadyForLSTM(64, model2, maxSequenceLen, XNewsSentences)
# X_news_LSTM -> total, 64
# _, X, Y, X_Next = createXForTimeSeries(ts, 100)

In [ ]:
X_news.shape

In [ ]:
# X_train, X_test, X_news_train, X_news_test, Y_train, Y_test, X_Next_train, X_Next_test = train_test_split(X, X_news_LSTM, Y, X_Next, test_size=0.2, random_state=42)
# X_train, X_test, X_news_train, X_news_test, Y_train, Y_test, X_Next_train, X_Next_test = train_test_split(X, X_news, Y, X_Next, test_size=0.2, random_state=42)
X_train, X_test, X_news_train, X_news_test, Y_train, Y_test, X_Next_train, X_Next_test = train_test_split(X, X_news_GRU, Y, X_Next, test_size=0.2, random_state=42)

In [ ]:
from torch.optim import Adam
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelTimeSeries = TimeSeries(1, 1, 768, 1) #input_size, output_size, hidden_dim, n_layers
optimizerT = Adam(modelTimeSeries.parameters(), lr=0.001)
criterionT = nn.MSELoss()
modelTimeSeries.to(device)
predicted_list = [random.randint(-20, 20) for _ in range(101)]
train_loader, test_loader = createDataLoaderForNews(X_train, X_news_train, Y_train, X_test, X_news_test, Y_test, 64, X_Next_train, X_Next_test)
evaluateTimeSeries(200, modelTimeSeries, optimizerT, train_loader, test_loader, criterionT)

In [ ]:
modelTimeSeries = TimeSeries(1, 1, 768, 1)
# torch.save(modelTimeSeries.state_dict(), '/content/drive/MyDrive/NLP Dataset/model/modelTimeSeries.pt')
modelTimeSeries.load_state_dict(torch.load('/content/drive/MyDrive/NLP Dataset/model/modelTimeSeries.pt'))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelTimeSeriesLSTM = TimeSeries(1, 1, 64, 1) #input_size, output_size, hidden_dim, n_layers
optimizerT = Adam(modelTimeSeriesLSTM.parameters(), lr=0.001)
criterionT = nn.MSELoss()
modelTimeSeriesLSTM.to(device)
train_loader, test_loader = createDataLoaderForNews(X_train, X_news_train, Y_train, X_test, X_news_test, Y_test, 64, X_Next_train, X_Next_test)
evaluateTimeSeries(200, modelTimeSeriesLSTM, optimizerT, train_loader, test_loader, criterionT)

In [ ]:
modelTimeSeriesLSTM = TimeSeries(1, 1, 64, 1)
# torch.save(modelTimeSeriesLSTM.state_dict(), '/content/drive/MyDrive/NLP Dataset/model/modelTimeSeriesLSTM.pt')
modelTimeSeriesLSTM.to(device)
modelTimeSeriesLSTM.load_state_dict(torch.load('/content/drive/MyDrive/NLP Dataset/model/modelTimeSeriesLSTM.pt'))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelTimeSeriesGRU = TimeSeries(1, 1, 64, 1) #input_size, output_size, hidden_dim, n_layers
optimizerT = Adam(modelTimeSeriesGRU.parameters(), lr=0.001)
criterionT = nn.MSELoss()
modelTimeSeriesGRU.to(device)
train_loader, test_loader = createDataLoaderForNews(X_train, X_news_train, Y_train, X_test, X_news_test, Y_test, 64, X_Next_train, X_Next_test)
evaluateTimeSeries(200, modelTimeSeriesGRU, optimizerT, train_loader, test_loader, criterionT)

In [ ]:
modelTimeSeriesGRU = TimeSeries(1, 1, 64, 1)

# torch.save(modelTimeSeriesGRU.state_dict(), '/content/drive/MyDrive/NLP Dataset/model/modelTimeSeriesGRU.pt')
modelTimeSeriesGRU.load_state_dict(torch.load('/content/drive/MyDrive/NLP Dataset/model/modelTimeSeriesGRU.pt'))

In [ ]:
def createChart(model, news, x, y, time):
  y_pred = model(x, news).detach()
  y = y.cpu().detach().numpy()
  
  pass

In [ ]:
from sklearn.metrics import r2_score
def calculateRSquare(test_loader, model):
  y_pred = []
  y_true = []
  for i, (x_batch, news_batch, y_batch, x_next) in enumerate(test_loader):
    y_predicted = model(x_batch, news_batch).detach()
    y_batch = y_batch.cpu().detach().numpy()
    y_predicted = y_predicted.cpu().numpy()
    for i in range(len(y_predicted)):
      y_true.append((y_batch[i]))
      y_pred.append((y_predicted[i][0]))
  print(r2_score(y_true, y_pred))

In [ ]:
calculateRSquare(test_loader, modelTimeSeriesGRU)

In [ ]:
calculateRSquare(test_loader, modelTimeSeriesLSTM)

In [ ]:
calculateRSquare(test_loader, modelTimeSeries)

In [ ]:
import seaborn as sns

def draw(y_actual, y_pred):
    num_rows = 101
    y_actual=y_actual.reshape(-1)
    y_pred=y_pred.reshape(-1)
    data_preproc = pd.DataFrame({'Days':[x for x in range(1, 102)], 'True Values': y_actual, 'Predicted Values': y_pred})
        
    sns.lineplot(x='Days', y='value', hue='variable', data=pd.melt(data_preproc, ['Days']))
def predict_graph(loader, scaler, model):
  for i, (x_batch, news_batch, y_batch, x_next) in enumerate(loader):
    index=60
    x=x_batch[index].unsqueeze(0)
    news=news_batch[index].unsqueeze(0)
    y_predicted_list = []
    y_new_pred = x_next[index].detach().cpu().numpy().reshape(-1)
    for k in range(100):
      y_pred=model(x, news)
      y_new_pre = y_pred.detach().cpu().numpy()
      y = y_new_pred[k]+predicted_list[k]
      y_predicted_list.append(y)
      x=x.detach().cpu().numpy()
      x=np.append(x, 1)
      x=x.reshape(1, -1)
      x=scaler.transform(x)
      x = x.reshape(-1, 1)
      x = x[:-1]
      x = np.append(x, y)
      x = x.reshape(1, -1)
      x=scaler.transform(x)
      x = x.reshape(-1, 1)
      x=x[1:]
      x=torch.tensor(x, dtype=torch.float32)
      x=x.unsqueeze(0)
      x=x.to(device)
    y_new_pred=np.append(y_new_pred, y_new_pred[-1])
    y_predicted_list.append(y_predicted_list[-1])
    draw(scaler.transform(y_new_pred.reshape(1, -1)), scaler.transform(np.array(y_predicted_list).reshape(1, -1))) 
    break
  
predict_graph(test_loader, scaler, modelTimeSeriesLSTM)

In [ ]:
#@title
predict_graph(test_loader, scaler, modelTimeSeriesGRU)